In [1]:
# Start from FileSHA in Threats data, then search into OTX Alien Vault and 'use tags to look into CVEs of Nist'


In [2]:
import datetime
import os
from OTXv2 import OTXv2
import time
import json 
import pandas as pd 
import requests
import urllib3
import json
import sys

pd.options.display.max_colwidth = 1000
pd.set_option('display.max_columns', None)

otx = OTXv2('276e51c0ef0cf3968203b219ac8bfe0a5bf68add0061c77ec1c118e7aac4e1fe')

In [3]:
threats_df = pd.read_csv(r'C:\Users\leona\OneDrive\Desktop\ThreatsData\cleaned\threats.csv', delimiter = ',', low_memory = False )
threats_df

,Unnamed: 0,ioc,ioc_type,threat_type,sha256_hash,sha3_384_hash,sha1_hash,md5_hash,source,first_seen,last_seen,file_name,file_size,file_type_mime,file_type,reporter,origin_country,signature,imphash,tlsh,telfhash,gimphash,ssdeep,tags,maliciousness,status,references
0,0,178.128.23.9:4125,ip:port,botnet_cc,NaN,NaN,NaN,NaN,Feodo Tracker,2021-05-16 19:49:33,2023-04-25,DIGITALOCEAN-ASN,NaN,NaN,NaN,NaN,SG,Dridex,NaN,NaN,NaN,NaN,NaN,NaN,NaN,online,NaN
1,1,104.248.178.90:4664,ip:port,botnet_cc,NaN,NaN,NaN,NaN,Feodo Tracker,2021-07-28 16:01:42,2023-04-25,DIGITALOCEAN-ASN,NaN,NaN,NaN,NaN,US,Dridex,NaN,NaN,NaN,NaN,NaN,NaN,NaN,online,NaN
2,2,192.99.150.39:7443,ip:port,botnet_cc,NaN,NaN,NaN,NaN,Feodo Tracker,2021-08-27 04:48:12,2023-04-25,OVH,NaN,ns509018.ip-192-99-150.net,NaN,NaN,CA,Dridex,NaN,NaN,NaN,NaN,NaN,NaN,NaN,online,NaN
3,3,128.199.232.159:6225,ip:port,botnet_cc,NaN,NaN,NaN,NaN,Feodo Tracker,2021-09-16 00:53:58,2023-04-25,DIGITALOCEAN-ASN,NaN,NaN,NaN,NaN,SG,Dridex,NaN,NaN,NaN,NaN,NaN,NaN,NaN,online,NaN
4,4,159.65.3.147:6225,ip:port,botnet_cc,NaN,NaN,NaN,NaN,Feodo Tracker,2021-09-17 11:44:00,2023-04-25,DIGITALOCEAN-ASN,NaN,NaN,NaN,NaN,SG,Dridex,NaN,NaN,NaN,NaN,NaN,NaN,NaN,online,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11147706,11147706,http://www.davidephoto.it/GsnIO/,url,NaN,d8ecfea0f73ad448bdd399894209d11542911bdd90dae5d324b00b5719cf1eb5,NaN,NaN,fc96d31aa6691dd0ae6b49e85b7e45f3,URLhaus,2018-03-05 14:26:22,NaN,NaN,NaN,NaN,exe,NaN,NaN,Heodo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11147707,11147707,https://girlhistory.ru/dUZfW/,url,NaN,d8ecfea0f73ad448bdd399894209d11542911bdd90dae5d324b00b5719cf1eb5,NaN,NaN,fc96d31aa6691dd0ae6b49e85b7e45f3,URLhaus,2018-03-05 14:21:09,NaN,NaN,NaN,NaN,exe,NaN,NaN,Heodo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11147708,11147708,http://sportists.com/Paid-Invoice/,url,NaN,477fed0554457b36c085b2e9174edd978e7fe537840eb793d038bec798f8e129,NaN,NaN,cc1924283df97f24500f5559c95098ca,URLhaus,2018-03-05 14:18:21,NaN,NaN,NaN,NaN,doc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11147709,11147709,http://hellohello-pension.com/Summit-Companies-Invoice-1451397/,url,NaN,477fed0554457b36c085b2e9174edd978e7fe537840eb793d038bec798f8e129,NaN,NaN,cc1924283df97f24500f5559c95098ca,URLhaus,2018-03-05 14:18:20,NaN,NaN,NaN,NaN,doc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
threats_df['ioc_type'].unique()

array(['ip:port', 'file', 'url', 'domain', 'sha256_hash', 'md5_hash'],
      dtype=object)

In [5]:
# Lookup of File SHA 256 in OTX Alien Vault

In [6]:
sha256_df = pd.DataFrame(threats_df[threats_df['ioc_type'] == 'sha256_hash'][['ioc','ioc_type','file_name','signature','source','first_seen', 'last_seen','origin_country','tags']],).reset_index(drop = True)
sha256_df.head(50)

,ioc,ioc_type,file_name,signature,source,first_seen,last_seen,origin_country,tags
0,ebeca2df24a55c629cf0ce0d4b703ed632819d8ac101b1b930ec666760036124,sha256_hash,win.conti,Conti,Threat Fox,2023-04-25 18:49:31 UTC,NaN,NaN,"['conti', 'Ransomware']"
1,f9d5b3567045a979512f89d9bbd6cd9d4a127c081590cc6cdae0b8cbf7a30215,sha256_hash,win.remcos,Remcos,Threat Fox,2023-04-25 18:45:47 UTC,NaN,NaN,"['RAT', 'remcos']"
2,7b63752ab40922b3a86a60bcf68b4676b6917958f5399c3543fa4eb0750f9be8,sha256_hash,win.remcos,Remcos,Threat Fox,2023-04-25 18:45:47 UTC,NaN,NaN,"['RAT', 'remcos']"
3,22bdea528c353ba3b53129ca5cd224a6e1a85faacfb968ab2c7b4ccfaaeb918c,sha256_hash,win.remcos,Remcos,Threat Fox,2023-04-25 18:45:47 UTC,NaN,NaN,"['RAT', 'remcos']"
4,fc5e7fc7fa6726c617fe423091e586193e356903db0ee9c6e0b464fcd29da37a,sha256_hash,win.remcos,Remcos,Threat Fox,2023-04-25 18:45:47 UTC,NaN,NaN,"['RAT', 'remcos']"
5,158aae7be7b74ab461bb3afcb61d9385f7122ae86c89a32b33312be7d7ce3ec3,sha256_hash,win.remcos,Remcos,Threat Fox,2023-04-25 18:45:47 UTC,NaN,NaN,"['RAT', 'remcos']"
6,936e38ad2ea282e761f352532a0e52f8994afcb880818145325f48538410e850,sha256_hash,win.remcos,Remcos,Threat Fox,2023-04-25 18:45:47 UTC,NaN,NaN,"['RAT', 'remcos']"
7,cec6baef4a0dcb3ff2abe34917278ee68b4656cad49c07d05d304a5566ca1e0f,sha256_hash,win.remcos,Remcos,Threat Fox,2023-04-25 18:45:47 UTC,NaN,NaN,"['RAT', 'remcos']"
8,9792958cff2d41bd7e00da58539ab10d8d9808494cb5a93cf0684720fc22c9e2,sha256_hash,win.remcos,Remcos,Threat Fox,2023-04-25 18:45:47 UTC,NaN,NaN,"['RAT', 'remcos']"
9,b4d144db0a51b75fec768e08ec6500afd3b085c0a9dd9b22926845536ec2a50d,sha256_hash,win.remcos,Remcos,Threat Fox,2023-04-25 18:45:47 UTC,NaN,NaN,"['RAT', 'remcos']"


In [7]:
otx_file_usefulColumns = ['otx_id','otx_name','otx_description', 'otx_tags', 'otx_attack_ids', 'otx_malware_families', 'otx_adversary', 'otx_modified', 'otx_created', 'otx_references', 'otx_author.username' ]

In [ ]:
# Example of an OTX ALien Vault request over a file malware { '../file/b501c7f2f1bc0b9cf39b65a08f24f8ac81564e4a8a6bc9dbe38e9fd8abfa60ec/general' }

In [13]:
ioc = 'ebeca2df24a55c629cf0ce0d4b703ed632819d8ac101b1b930ec666760036124'

current_pulse = otx.get('https://otx.alienvault.com/api/v1/indicators/file/'+ ioc +'/general')
current_pulse



{'sections': ['general', 'analysis'],
 'type': 'sha256',
 'type_title': 'FileHash-SHA256',
 'indicator': 'ebeca2df24a55c629cf0ce0d4b703ed632819d8ac101b1b930ec666760036124',
 'validation': [],
 'base_indicator': {'id': 2762178836,
  'indicator': 'ebeca2df24a55c629cf0ce0d4b703ed632819d8ac101b1b930ec666760036124',
  'type': 'FileHash-SHA256',
  'title': '',
  'description': '',
  'content': '',
  'access_type': 'public',
  'access_reason': ''},
 'pulse_info': {'count': 6,
  'pulses': [{'id': '622b1b519fa899168e2665ca',
    'name': 'Threat Profile: Conti Ransomware',
    'description': "A new ransomware family known as Conti has been discovered using multiple techniques to encrypt files and encrypt them for the decryption key, according to research by McAfee's MVISion Insights.",
    'modified': '2022-04-28T15:26:50.727000',
    'created': '2022-03-11T09:50:09.760000',
    'tags': ['conti',
     'cobalt strike',
     'mitre att',
     'response',
     'rapid response',
     'sophos rapid',

In [14]:
import json 
with open("OTXAlienVault_sample.json", 'w') as f:
    json.dump(current_pulse, f)

In [9]:
ioc = 'b501c7f2f1bc0b9cf39b65a08f24f8ac81564e4a8a6bc9dbe38e9fd8abfa60ec'

current_pulse = otx.get('https://otx.alienvault.com/api/v1/indicators/file/'+ ioc +'/analysis')
current_pulse
#tmp = pd.DataFrame.from_dict(pd.json_normalize(current_pulse))
#tmp
#otxAlienVault_df = pd.DataFrame.from_dict(pd.json_normalize(tmp['pulse_info.pulses'])).T


#otxAlienVault_df = pd.DataFrame.from_dict(pd.json_normalize(otxAlienVault_df[0])).add_prefix('otx_')
#otxAlienVault_df

{'analysis': {}, 'page_type': 'generic', 'malware': {}}

In [9]:
pd.DataFrame(sha256_df[sha256_df.index == 31])


,ioc,ioc_type,file_name,signature,source,first_seen,last_seen,origin_country,tags
31,b501c7f2f1bc0b9cf39b65a08f24f8ac81564e4a8a6bc9dbe38e9fd8abfa60ec,sha256_hash,win.smokeloader,SmokeLoader,Threat Fox,2023-04-24 22:07:35 UTC,NaN,NaN,"['plugins', 'smokeloader', 'stage2']"


In [10]:
def getFileEntriesFromOTXAlienVault(df):
    
    otx_file_usefulColumns = ['otx_id','otx_name','otx_description', 'otx_tags', 'otx_attack_ids', 'otx_malware_families', 'otx_adversary', 'otx_modified', 'otx_created', 'otx_references', 'otx_author.username' ]
    
    final_df = pd.DataFrame()
    
    useful_columns = df.columns
        
    for i in range(len(df)):
                
        ioc = df.at[i,'ioc']
        el = pd.DataFrame(df[df.index == i])

        print(i)
        
        current_pulse = otx.get('https://otx.alienvault.com/api/v1/indicators/file/'+ ioc +'/general')
        tmp = pd.DataFrame.from_dict(pd.json_normalize(current_pulse)).reset_index(drop = True)        
        otxAlienVault_df = pd.DataFrame.from_dict(pd.json_normalize(tmp['pulse_info.pulses'])).T
        
        if(otxAlienVault_df.empty == False):
        
            otxAlienVault_df = pd.DataFrame.from_dict(pd.json_normalize(otxAlienVault_df[0])).add_prefix('otx_')
            #print('\n')
            
            #print(len(otxAlienVault_df))
            
            rows_otx = pd.DataFrame()
            rows_otx = otxAlienVault_df[otx_file_usefulColumns]
            rows_threats = pd.DataFrame()
            rows_threats = el[useful_columns]

            
            
            rows = pd.concat([rows_threats,rows_otx], axis = 1)
            
            #print(len(rows))
            rows[useful_columns] = rows[useful_columns].fillna(method='ffill')

            
            if(len(final_df) > 1):
                rows = rows.iloc[1:, :]

            #print(len(rows))
            
            #print('\n')
            
            final_df = pd.concat([final_df,rows], ignore_index = True)
   
    return final_df

In [11]:
OTXAlienVault_FileSHA_df = getFileEntriesFromOTXAlienVault(sha256_df)
OTXAlienVault_FileSHA_df

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

,ioc,ioc_type,file_name,signature,source,first_seen,last_seen,origin_country,tags,otx_id,otx_name,otx_description,otx_tags,otx_attack_ids,otx_malware_families,otx_adversary,otx_modified,otx_created,otx_references,otx_author.username
0,ebeca2df24a55c629cf0ce0d4b703ed632819d8ac101b1b930ec666760036124,sha256_hash,win.conti,Conti,Threat Fox,2023-04-25 18:49:31 UTC,NaN,NaN,"['conti', 'Ransomware']",622b1b519fa899168e2665ca,Threat Profile: Conti Ransomware,"A new ransomware family known as Conti has been discovered using multiple techniques to encrypt files and encrypt them for the decryption key, according to research by McAfee's MVISion Insights.","[conti, cobalt strike, mitre att, response, rapid response, sophos rapid, conti group, sophos, rclone, instrumentation, mega, icmp, next, back, public, github, yararules, jump, strong, sign, view, code issues, pull, wiki security, copy, contact, star, ransomware, ransom, unknown, sodinokibi, ryuk, sqlagent, mssql, service, mssqlfdlauncher, windows, sql2008 y, systembgc y, tps y, tpsama y, chacha, noise, ryuk ransomware, magic, rule id, ransomwarehau, containment, atp rules, rp dynamic, new startup, protection, avengine v2, avengine v3, Wizard Spider]","[{'id': 'T1011', 'name': 'Exfiltration Over Other Network Medium', 'display_name': 'T1011 - Exfiltration Over Other Network Medium'}, {'id': 'T1016', 'name': 'System Network Configuration Discovery', 'display_name': 'T1016 - System Network Configuration Discovery'}, {'id': 'T1018', 'name': 'Remote System Discovery', 'display_name': 'T1018 - Remote System Discovery'}, {'id': 'T1021', 'name': 'Remote Services', 'display_name': 'T1021 - Remote Services'}, {'id': 'T1036', 'name': 'Masquerading', 'display_name': 'T1036 - Masquerading'}, {'id': 'T1046', 'name': 'Network Service Scanning', 'display_name': 'T1046 - Network Service Scanning'}, {'id': 'T1047', 'name': 'Windows Management Instrumentation', 'display_name': 'T1047 - Windows Management Instrumentation'}, {'id': 'T1055', 'name': 'Process Injection', 'display_name': 'T1055 - Process Injection'}, {'id': 'T1059', 'name': 'Command and Scripting Interpreter', 'display_name': 'T1059 - Command and Scripting Interpreter'}, {'id': 'T1078'...","[{'id': 'Cobalt Strike', 'display_name': 'Cobalt Strike', 'target': None}, {'id': 'Ryuk', 'display_name': 'Ryuk', 'target': None}, {'id': 'Sodinokibi', 'display_name': 'Sodinokibi', 'target': None}, {'id': 'Conti', 'display_name': 'Conti', 'target': None}]",Conti,2022-04-28T15:26:50.727000,2022-03-11T09:50:09.760000,"[https://kc.mcafee.com/corporate/index?page=content&id=KB93317, https://www.carbonblack.com/blog/tau-threat-discovery-conti-ransomware/, https://github.com/advanced-threat-research/Yara-Rules/blob/master/ransomware/Ransom_Conti.yar, https://news.sophos.com/en-us/2021/02/16/conti-ransomware-attack-day-by-day/, https://www.clearskysec.com/conti-ransomware/]",BITSecurity
1,ebeca2df24a55c629cf0ce0d4b703ed632819d8ac101b1b930ec666760036124,sha256_hash,win.conti,Conti,Threat Fox,2023-04-25 18:49:31 UTC,NaN,NaN,"['conti', 'Ransomware']",609bcbfa86695a9f1f3c2784,Conti Ransomware -- (20/12/2021 updated for Log4Shell integration),uses multiple threads to encrypt files at a faster rate compared to other ransomware families and contains command-line options to scan for local files as well as remote files over SMB shares. Conti also uses the Windows Restart Manager to free up files that are open by various applications. The ransomware uses AES-256 encryption and requires the victim to email the threat actor for the decryption key. Variants of the malware post stolen data from entities who refuse to pay the ransom.\n\n\nEdit: As of writing (20/12/2021) the Vulnerability Log4Shell has been integrated with Conti Ransomware as a handler.,"[Conti Ransomware, Conti, Ransomware, AES-256, SMB, CVE-2021-44228, Log4Shell, Log4j]","[{'id': 'T1190', 'name': 'Exploit Public-Facing Application', 'display_name': 'T1190 - Exploit Public-Facing Application'}, {'id': 'T1086'

In [12]:
OTXAlienVault_FileSHA_df.tail(50)

,ioc,ioc_type,file_name,signature,source,first_seen,last_seen,origin_country,tags,otx_id,otx_name,otx_description,otx_tags,otx_attack_ids,otx_malware_families,otx_adversary,otx_modified,otx_created,otx_references,otx_author.username
250,dfa2c6b121928f5e6263831ea7a6ce2afc2d5e3d98f752f0ff9cfea94871e8c3,sha256_hash,win.smokeloader,SmokeLoader,Threat Fox,2023-04-21 15:32:50 UTC,NaN,NaN,"['plugins', 'smokeloader', 'stage2']",644adffc006db6bd30913332,SmokeLoader Plugin and Second Stage Hashes 20.04.2023 - 27.04.2023,This is a list of SmokeLoader infection related artifacts collected during the last week. If you encounter any FP please contact me via https://twitter.com/gi7w0rm :),"[SmokeLoader, 2nd_stage, plugins]",[],"[{'id': 'ALF:Trojan:Win32/SmokeLoader', 'display_name': 'ALF:Trojan:Win32/SmokeLoader', 'target': None}, {'id': 'SmokeLoader', 'display_name': 'SmokeLoader', 'target': None}, {'id': 'Trojan:Win32/SmokeLoader', 'display_name': 'Trojan:Win32/SmokeLoader', 'target': '/malware/Trojan:Win32/SmokeLoader'}]",,2023-04-27T20:50:04.056000,2023-04-27T20:50:04.056000,[https://raw.githubusercontent.com/Gi7w0rm/MalwareConfigLists/main/SmokeLoader/hashes/sha265_second_stage_and_plugins.txt],@Gi7w0rm
251,a0a1d60b2fc491e1c6c2a8e5db1d5566297926e5f97c06c71a0c10cc364ef083,sha256_hash,win.smokeloader,SmokeLoader,Threat Fox,2023-04-21 15:32:50 UTC,NaN,NaN,"['plugins', 'smokeloader', 'stage2']",644adffc006db6bd30913332,SmokeLoader Plugin and Second Stage Hashes 20.04.2023 - 27.04.2023,This is a list of SmokeLoader infection related artifacts collected during the last week. If you encounter any FP please contact me via https://twitter.com/gi7w0rm :),"[SmokeLoader, 2nd_stage, plugins]",[],"[{'id': 'ALF:Trojan:Win32/SmokeLoader', 'display_name': 'ALF:Trojan:Win32/SmokeLoader', 'target': None}, {'id': 'SmokeLoader', 'display_name': 'SmokeLoader', 'target': None}, {'id': 'Trojan:Win32/SmokeLoader', 'display_name': 'Trojan:Win32/SmokeLoader', 'target': '/malware/Trojan:Win32/SmokeLoader'}]",,2023-04-27T20:50:04.056000,2023-04-27T20:50:04.056000,[https://raw.githubusercontent.com/Gi7w0rm/MalwareConfigLists/main/SmokeLoader/hashes/sha265_second_stage_and_plugins.txt],@Gi7w0rm
252,5339bda196e6090309995c18f83fab58d570fa3564f5c6afae9b72ca9f66140f,sha256_hash,win.smokeloader,SmokeLoader,Threat Fox,2023-04-21 15:32:50 UTC,NaN,NaN,"['plugins', 'smokeloader', 'stage2']",644adffc006db6bd30913332,SmokeLoader Plugin and Second Stage Hashes 20.04.2023 - 27.04.2023,This is a list of SmokeLoader infection related artifacts collected during the last week. If you encounter any FP please contact me via https://twitter.com/gi7w0rm :),"[SmokeLoader, 2nd_stage, plugins]",[],"[{'id': 'ALF:Trojan:Win32/SmokeLoader', 'display_name': 'ALF:Trojan:Win32/SmokeLoader', 'target': None}, {'id': 'SmokeLoader', 'display_name': 'SmokeLoader', 'target': None}, {'id': 'Trojan:Win32/SmokeLoader', 'display_name': 'Trojan:Win32/SmokeLoader', 'target': '/malware/Trojan:Win32/SmokeLoader'}]",,2023-04-27T20:50:04.056000,2023-04-27T20:50:04.056000,[https://raw.githubusercontent.com/Gi7w0rm/MalwareConfigLists/main/SmokeLoader/hashes/sha265_second_stage_and_plugins.txt],@Gi7w0rm
253,ddb78004df154ac51bf30802b647dabb5479f1769d8eb3279cbfad1ee7bce481,sha256_hash,win.smokeloader,SmokeLoader,Threat Fox,2023-04-21 15:32:50 UTC,NaN,NaN,"['plugins', 'smokeloader', 'stage2']",644adffc006db6bd30913332,SmokeLoader Plugin and Second Stage Hashes 20.04.2023 - 27.04.2023,This is a list of SmokeLoader infection related artifacts collected during the last week. If you encounter any FP please contact me via https://twitter.com/gi7w0rm :),"[SmokeLoader, 2nd_stage, plugins]",[],"[{'id': 'ALF:Trojan:Win32/SmokeLoader', 'display_name': 'ALF:Trojan:Win32/SmokeLoader', 'target': None}, {'id': 'SmokeLoader', 'display_name': 'SmokeLoader', 'target': None}, {'id': 'Trojan:Win32/SmokeLoader', 'display_name': 'Trojan:Win32/SmokeLoader', 'target': '/malware/Trojan:Win32/SmokeLoader'}]",,2023-04-27T20:50:04.056000,2023-0

In [14]:
OTXAlienVault_FileSHA_df.to_csv(r'C:\Users\leona\OneDrive\Desktop\ThreatsData\cleaned\OTXAlienVault_lookups\OTXAlienVault_FileSHA_df.csv')